# Example code for backend module tensor_network.node_array

**This example code is only of interest to time_evolving_mpo developers as it concerns the inner working of the backend.**

In [1]:
import sys
sys.path.insert(0,'../..')

import numpy as np
import tensornetwork as tn
import time_evolving_mpo.backends.tensor_network.node_array as node_array

In [2]:
tn.__version__

'0.3.0'

## Create

In [3]:
np.random.seed(0)
a = np.random.rand(3,4,9,5)
b = np.random.rand(5,5,9,6)
c = np.random.rand(6,5,9,2)
d = np.random.rand(2,3,9)

In [4]:
na = node_array.NodeArray([a,b,c,d], right=False, name="my node array")
print(na)

my node array:
  ~~[3, 4, 9, 5]~~~~[5, 5, 9, 6]~~~~[6, 5, 9, 2]~~~~[2, 3, 9]  


In [5]:
print(na.bond_dimensions)
print(na.rank)

[5, 6, 2]
2


In this context the `rank` of a NodeArray is the number of danglig 'physical' legs per node. For example, the `rank` of an MPS is 1, the `rank` of an MPO is 2, and so on. Also, an NodeArray can have or not have left and right dangling legs.

In [6]:
dd = node_array.NodeArray([d], name="D1", left=False, right=False)
print(dd)
print(f"    rank = {dd.rank}\n")

dd = node_array.NodeArray([d], name="D2", left=False, right=True)
print(dd)
print(f"    rank = {dd.rank}\n")

dd = node_array.NodeArray([d], name="D3", left=True, right=False)
print(dd)
print(f"    rank = {dd.rank}\n")

dd = node_array.NodeArray([d], name="D4", left=True, right=True)
print(dd)
print(f"    rank = {dd.rank}\n")

D1:
    [2, 3, 9]  
    rank = 3

D2:
    [2, 3, 9]~~
    rank = 2

D3:
  ~~[2, 3, 9]  
    rank = 2

D4:
  ~~[2, 3, 9]~~
    rank = 1



## Apply matrix (left/right)

In [7]:
m = np.random.rand(3,7)
print(na)
na.apply_matrix(m,left=True)
print(na)

my node array:
  ~~[3, 4, 9, 5]~~~~[5, 5, 9, 6]~~~~[6, 5, 9, 2]~~~~[2, 3, 9]  
my node array:
  ~~[7, 4, 9, 5]~~~~[5, 5, 9, 6]~~~~[6, 5, 9, 2]~~~~[2, 3, 9]  


## Apply vector (left/right)

In [8]:
v = np.random.rand(7)
print(na)
na.apply_vector(v, left=True)
print(na)

my node array:
  ~~[7, 4, 9, 5]~~~~[5, 5, 9, 6]~~~~[6, 5, 9, 2]~~~~[2, 3, 9]  
my node array:
    [4, 9, 5]~~~~[5, 5, 9, 6]~~~~[6, 5, 9, 2]~~~~[2, 3, 9]  


## Join

In [9]:
x = np.random.rand(2,9,3)
na0 = node_array.NodeArray([x], left=False, name="array 0")
na1 = node_array.NodeArray([a,b], name="array 1")
na2 = node_array.NodeArray([c,d], right=False, name="array 2")
print(na0)
print(na1)
print(na2)

array 0:
    [2, 9, 3]~~
array 1:
  ~~[3, 4, 9, 5]~~~~[5, 5, 9, 6]~~
array 2:
  ~~[6, 5, 9, 2]~~~~[2, 3, 9]  


In [10]:
na_12 = node_array.join(na1,na2)
print(na_12)

__unnamed__:
  ~~[3, 4, 9, 5]~~~~[5, 5, 9, 6]~~~~[6, 5, 9, 2]~~~~[2, 3, 9]  


In [11]:
na_012 = node_array.join(na0, na_12)
print(na_012)

__unnamed__:
    [2, 9, 3]~~~~[3, 4, 9, 5]~~~~[5, 5, 9, 6]~~~~[6, 5, 9, 2]~~~~[2, 3, 9]  


## Split

In [12]:
na_A, na_B = node_array.split(na_012, 2)

In [13]:
print(na_A)
print(na_B)

__unnamed__:
    [2, 9, 3]~~~~[3, 4, 9, 5]~~
__unnamed__:
  ~~[5, 5, 9, 6]~~~~[6, 5, 9, 2]~~~~[2, 3, 9]  


## SVD sweep

In [14]:
print(na_012)

__unnamed__:
    [2, 9, 3]~~~~[3, 4, 9, 5]~~~~[5, 5, 9, 6]~~~~[6, 5, 9, 2]~~~~[2, 3, 9]  


In [15]:
na_012.svd_sweep(1, 3, max_singular_values=4)
print(na_012)

__unnamed__:
    [2, 9, 3]~~~~[3, 4, 9, 4]~~~~[4, 5, 9, 4]~~~~[4, 5, 9, 2]~~~~[2, 3, 9]  


In [16]:
singular_values = na_012.svd_sweep(-1, 0, max_singular_values=1)
print(na_012)
print("Singular values:")
for keep, discard in singular_values:
    print(f"  [keep / discard]: {keep} / {discard}")

__unnamed__:
    [2, 9, 1]~~~~[1, 4, 9, 1]~~~~[1, 5, 9, 1]~~~~[1, 5, 9, 1]~~~~[1, 3, 9]  
Singular values:
  [keep / discard]: [3.99430628] / [1.49866144]
  [keep / discard]: [10599.90104242] / [267.0975746  201.42531644 171.11795386]
  [keep / discard]: [10587.82415649] / [339.6380433  304.27467941 218.9626894 ]
  [keep / discard]: [10310.27450257] / [1745.62805973 1659.22955843]


## Zip-up contract

### 1st example

In [17]:
mps = node_array.NodeArray([np.random.rand(3,2,3),
                            np.random.rand(3,3,3),
                            np.random.rand(3,4,3)],
                           name="MPS")
print(mps)
print(f"    rank:{mps.rank}")

mpo1 = node_array.NodeArray([np.random.rand(2,5,4),
                             np.random.rand(4,3,5,4),
                             np.random.rand(4,4,5)], 
                            left=False, 
                            right=False, 
                            name="MPO1")
print(mpo1)
print(f"    rank:{mpo1.rank}")

mpo2 = node_array.NodeArray([np.random.rand(5,5,3),
                             np.random.rand(3,5,5,3),
                             np.random.rand(3,5,5)], 
                            left=False, 
                            right=False, 
                            name="MPO2")
print(mpo2)
print(f"    rank:{mpo2.rank}")


MPS:
  ~~[3, 2, 3]~~~~[3, 3, 3]~~~~[3, 4, 3]~~
    rank:1
MPO1:
    [2, 5, 4]~~~~[4, 3, 5, 4]~~~~[4, 4, 5]  
    rank:2
MPO2:
    [5, 5, 3]~~~~[3, 5, 5, 3]~~~~[3, 5, 5]  
    rank:2


In [18]:
mps.zip_up(mpo1, [(0, 0)], left_index=0, right_index=-1, max_singular_values=10)
print(mps)

MPS:
  ~~[3, 5, 10]~~~~[10, 5, 10]~~~~[10, 5, 3]~~


In [19]:
mps.zip_up(mpo2, [(0, 0)], left_index=0, max_singular_values=11)
print(mps)

MPS:
  ~~[3, 5, 11]~~~~[11, 5, 11]~~~~[11, 5, 3]~~


In [20]:
mps.svd_sweep(-1,0,max_singular_values=2);
print(mps)

MPS:
  ~~[3, 5, 2]~~~~[2, 5, 2]~~~~[2, 5, 3]~~


### 2nd example

In [21]:
mps = node_array.NodeArray([np.random.rand(4,3),
                            np.random.rand(3,4,3),
                            np.random.rand(3,4)],
                           left=False,
                           right=False,
                           name="MPS")
mps1 = mps.copy()
mps2 = mps.copy()
mps3 = mps.copy()
mps4 = mps.copy()
mps5 = mps.copy()
mpsL = node_array.NodeArray([np.random.rand(3,4,3),
                             np.random.rand(3,4)],
                            left=True,
                            right=False,
                            name="MPS")
print(mps)
print(f"    rank:{mps.rank}")
print(mpsL)
print(f"    rank:{mpsL.rank}")

mpo1 = node_array.NodeArray([np.random.rand(3,4,4,3),
                             np.random.rand(3,4,4)], 
                            left=True, 
                            right=False, 
                            name="MPO1")
mpo5 = mpo1.copy()
print(mpo1)
print(f"    rank:{mpo1.rank}")

mpo2 = node_array.NodeArray([np.random.rand(4,4,3),
                             np.random.rand(3,4,4,3)], 
                            left=False, 
                            right=True, 
                            name="MPO2")
print(mpo2)
print(f"    rank:{mpo2.rank}")

arr1 = node_array.NodeArray([np.random.rand(4,2,2,3),
                             np.random.rand(3,4,2,2,3),
                             np.random.rand(3,4,2,2,3)], 
                            left=False, 
                            right=True, 
                            name="array1")
print(arr1)
print(f"    rank:{arr1.rank}")

arr2 = node_array.NodeArray([np.random.rand(3,2,4,2,3),
                             np.random.rand(3,2,4,2,3),
                             np.random.rand(3,2,4,2)], 
                            left=True, 
                            right=False, 
                            name="array2")
print(arr2)
print(f"    rank:{arr2.rank}")

MPS:
    [4, 3]~~~~[3, 4, 3]~~~~[3, 4]  
    rank:1
MPS:
  ~~[3, 4, 3]~~~~[3, 4]  
    rank:1
MPO1:
  ~~[3, 4, 4, 3]~~~~[3, 4, 4]  
    rank:2
MPO2:
    [4, 4, 3]~~~~[3, 4, 4, 3]~~
    rank:2
array1:
    [4, 2, 2, 3]~~~~[3, 4, 2, 2, 3]~~~~[3, 4, 2, 2, 3]~~
    rank:3
array2:
  ~~[3, 2, 4, 2, 3]~~~~[3, 2, 4, 2, 3]~~~~[3, 2, 4, 2]  
    rank:3


In [22]:
mps1.zip_up(mpo1, [(0, 0)], left_index=0, direction="left")
print(mps1)

MPS:
  ~~[3, 4, 9]~~~~[9, 4, 3]~~~~[3, 4]  


In [23]:
mps2.zip_up(mpo2, [(0, 0)], left_index=1, right_index=2, direction="left")
print(mps2)

MPS:
    [4, 3]~~~~[3, 4, 9]~~~~[9, 4, 3]~~


In [24]:
mps3.zip_up(arr1, [(0, 0)])
print(mps3)

MPS:
    [2, 2, 4]~~~~[4, 2, 2, 9]~~~~[9, 2, 2, 3]~~


In [25]:
mps3.zip_up(arr2, [(0, 0),(1, 2)], right_index=-1, direction="left", max_singular_values=7)
print(mps3)

MPS:
  ~~[3, 4, 7]~~~~[7, 4, 7]~~~~[7, 4, 3]~~


In [26]:
mps4.contract(mps, [(0,0)], direction="right")
print(mps4)
print(mps4.get_nodes())

MPS:
    []  
[Node
(
name : '__unnamed_node__',
tensor : 
array(152.42919023),
edges : 
[] 
)]


In [27]:
print(mps1)
print(mps)
mps1.contract(mps, [(0,0)])
print(mps1)
mps1.get_nodes()

MPS:
  ~~[3, 4, 9]~~~~[9, 4, 3]~~~~[3, 4]  
MPS:
    [4, 3]~~~~[3, 4, 3]~~~~[3, 4]  
MPS:
  ~~[3]  


[Node
 (
 name : '__unnamed_node__',
 tensor : 
 array([1601.45765138, 1707.79167513, 1399.5037339 ]),
 edges : 
 [
 Edge(Dangling Edge)[0] 
 ] 
 )]

In [28]:
# mps5.contract(mpsL, [(0,0)], left_index=0, direction="left")

In [29]:
mps5.contract(mpsL, [(0,0)], left_index=0, direction="right")
print(mps5)

MPS:
  ~~[3, 4]  
